# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd # Вызываем бибилиотеку Pandas

from pymystem3 import Mystem # Вызываем лемматизатор для слов на русском
m = Mystem()

from collections import Counter # Вызываем контейнер Counter

df = pd.read_csv('/datasets/data.csv') # Читаем данные
 
df.info() # Смотрим общую информацию о таблице

print(df.columns) # Посмотрим названия столбцов на наличие ошибок

df.head(10) # Посмотрим первые 10 строк таблицы



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод** При изучении общей информации выявлены следующие недочеты в данных:
1. Пустые строки в колонках days_employed (трудовой стаж в днях) и total_income (доход в месяц)
2. Отрицательные значения колонке в days_employed (трудовой стаж в днях), что нелогично
3. Разное написание (по регистру букв) типа образования
4. Неверный тип данных в колонках: days_employed (трудовой стаж в днях) - логичнее int64; 
5. Интересное наблюдение - в таблице назван dob_years, а в описании данных dob_days — возраст клиента в годах (причем только в pdf файле). Поскольку в реальной таблице названо даже правильней, я бы наверное обратился к тем кто давал данные с предложением поменять описание

## Шаг 2. Предобработка данных

### Обработка пропусков

####  Обработка столбца days_employed

In [2]:
print('Максимальный стаж -', df['days_employed'].max()/248, 'лет', 'Медианный стаж -', df['days_employed'].median()/248, 'лет', 'Минимальный стаж -', df['days_employed'].min()/248, 'лет' ) #Смотрю мин/макс/медиану - очень большой разброс значений, заполнять пропущенный значения осредненными ошибочно

print('Наличие записей с нулевым стажем -', df[df['days_employed'] == 0])# Проверяю наличие записей с нулевым стажем

print('Пустых значений в days_employed', df['days_employed'].isna().sum()) #Считаем общее количество пустых значений

df['days_employed'] = df['days_employed'].fillna(0) # Заполняю пропущенные значения в 'days_employed' - значанием 0


Максимальный стаж - 1619.9814535295566 лет Медианный стаж - -4.852296486977779 лет Минимальный стаж - -74.14899153454994 лет
Наличие записей с нулевым стажем - Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []
Пустых значений в days_employed 2174


####  Обработка столбца total_income

In [3]:
print('Максимальный доход - ', df['total_income'].max(), 'Медианный доход - ', df['total_income'].median(), 'Минимальный доход -', df['total_income'].min() ) #Смотрю мин/макс/медиану - очень большой разброс значений, заполнять пропущенный значения осредненными ошибочно

print('Пустых значений в days_employed', df['days_employed'].isna().sum()) #Смотрю общее количество пустых значений


df['total_income'] = df['total_income'].fillna(0)  # Заполняю пропущенные значения в 'total_income' - значением 0

df.info() # Проверяем, все ли заполнено.

Максимальный доход -  2265604.028722744 Медианный доход -  145017.93753253992 Минимальный доход - 20667.26379327158
Пустых значений в days_employed 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Пропущенные значения в колонках 'days_employed' и 'total_income' заполнил 0-ми значениями. Заполнение пропусков осредненными значениями с моей точки зрения может привести к существенному искажению данных.
В колонке 'total_income' пропущенные значения скорее всего говорят об отсутствии дохода (либо как вариант - отутствие подтвержденного дохода), поскольку анализ данных до замещения пропусков на значение '0' показал, что нулевых значений не было в принципе, что нелогично. Также в момент оценки кредитоспособности заемщика предполагать зарабатываемую им сумму опасно. Еще один аргумент за то, чтобы не заполнять осредненными/медианными значениями - очень большой разброс значений минимального и максимального дохода, что говорит о большой неоднородности группы заемщиков.

В колонке 'days_employed' пропущенные значения скорее всего говорят о некорректном заполнении таблицы, что косвенно подтверждается наличием данных о отрицательном стаже, а также максимальным значение стажа 1620 лет. Как и в предыдущем случае - анализ данных до замещения пропусков на значение '0' показал, что нулевых значений не было, что  на мой взгляд странно, хотя опять же это может говорить об отсутствии подтвержденного стажа. Если начать осреднять, то можно получить в некоторых строках стаж превышающий возраст, что внесет еще больше разнообразия. Поскольку данный столбец дальше не используется для анализа данных, думаю дальнейший его анализ нецелесообразен. В реальной ситуации я бы попробовал запросить тех кто его выгружал - на предмет отрицательных и нулевых значений.


### Замена типа данных

#### Замена типа данных в days_employed

In [4]:
df['days_employed'] = df['days_employed'].astype('int') #Заменяем тип данных на int

df['days_employed'] = df['days_employed'].abs()  # Убираем отрицательные значения, хотя в данном случае в анализе эти данные не используются, внутренний перфекционист настаивает :)


#### Замена типа данных в total_income

In [5]:
df['total_income'] = df['total_income'].astype('int') #Заменяем тип данных на int

**Вывод** Заменили в двух стобцах ('days_employed' и 'total_income') тип переменных с float64 на int64, потому как что в том что в другом случае наличие знаков после запятой никак не повлияет на результат анализа, но сильно сказывается на наглядности таблицы. 
Замена произведена методом astype, потому что метод позволяет выбрать тип данных, в который переводим. (to_numeric - переводит только из строк в вещественный тип) 
Убраны отрицательные значения из 'days_employed' и 'children'. На мой взгляд и там и там это является ошибкой ввода или выгрузки данных. И если 'days_employed' для ответа на поставленные вопросы использоваться не будет, то 'children' будет и наличие отрицательного количества детей будет неправильно.

#### Исключение отрицательных величин в children

In [6]:
df['children'] = df['children'].abs() # Убираем отрицательное количество детей.

df.head()  #Проверяем результат


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Обработка дубликатов

#### Приведение education к единому написанию

In [7]:
print(df['education'].value_counts()) # Перед тем как искать дубликаты, посмотрим колонку 'education' на предмет уникальных значений

df['education'] = df['education'].str.lower() #Приводим к единому написанию



среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


#### Удаление дубликатов

In [8]:
df.duplicated().sum() # Посмотрим количество дубликатов в датафрейме


df = df.drop_duplicates().reset_index() # Удалим дубликаты с переписыванием индексов

df.info() #Проверим результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
index               21454 non-null int64
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(8), object(5)
memory usage: 2.1+ MB


**Вывод** Перед поиском дубликатов привел к единому написанию столбец 'education'. Методом duplicated().sum() нашли  дубликаты и посчитали их количество. 
Учитывая, что:
- их общее количество равно 71, 
- совпадения есть по двум количественным параметрам, а именно - days_employed и total_income, вероятность совпадения которых крайне мала, 
- общее количество строк 21 525, а соответственно 71 строка это 0,33% от общего количества

приходим к выводу, что что все дубликаты можно удалять. 

Для удаления применяем метод drop_duplicates().reset_index() с перезаписыванием индексов.

### Лемматизация

In [9]:
#def lemm_purp (column):         # Пишем функцию для лемматизации строк
#    lemmas = m.lemmatize(column)
#    return lemmas
df['lemmatize_purpose'] = df['purpose'].apply(m.lemmatize)
 # Добавляем в датафрейм колонку с лемматизированными строками

# print(df.head()) # Для себя проверял датафрейм на успешность добавления столбца

lemm_for_count =[] # Определяем переменную для списка
for j in df['lemmatize_purpose']: # Создаем список из списка списков
    lemm_for_count.extend(j)

Counter(lemm_for_count).most_common() # Подсчитаем встречаемость значений




[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

**Вывод** Лемматизируем столбец purpose для дальнейшей категоризации и анализа данных, для чего создаем функцию для перебора строк. 
Результат лемматизации добавляем в новый столбец.
Методом Counter смотрим какие варианты целей кредитов существуют. 
Из полученных данных можно сделать вывод, что основных целей кредита 5: 'приобретение объекта недвижимости', 'приобретение автомобиля', 'кредит на образование', 'кредит на свадьбу', 'кредит на ремонт'

### Категоризация данных

#### Категоризация по доходу

In [10]:
median = df['total_income'].median() # Определяем медианную зарплату
lower_bound = median - median * 0.5 # Определяем нижнюю границу - медианная - 50%
upper_bound = median + median * 0.5 # Определяем верхнюю границу - медианная + 50%

#print(median, lower_bound, upper_bound) # Для себя проверил результат вычисления

def income(tot_inc):   # Создаем функцию для категоризации по размеру дохода
    if tot_inc == 0:
        return 'без дохода'
    if tot_inc <= lower_bound:
        return 'низкий'
    if lower_bound < tot_inc <= upper_bound:
        return 'средний'
    if tot_inc > upper_bound:
        return 'высокий'

df['total_income_category'] = df['total_income'].apply(income) # Создаем новый столбец с категоризацией по доходу

#### Категоризация по цели кредита

In [11]:
def purpose_name(purp):  # Создаем функцию для категоризации по цели кредита
    try:
        if 'ремонт' in purp:
            return 'кредит на ремонт'
        if 'жилье' in purp:
            return 'приобретение объекта недвижимости'
        if 'недвижимость' in purp:
            return 'приобретение объекта недвижимости'
        if 'жилой' in purp:
            return 'приобретение объекта недвижимости'
        if 'автомобиль' in purp:
            return 'приобретение автомобиля'
        if 'образование' in purp:
            return 'кредит на образование'
        if 'свадьба' in purp:
            return 'кредит на свадьбу'
    except:
        return 'цель кредита не определилась леммами надо проверить'
    
df['purpose_category'] = df['lemmatize_purpose'].apply(purpose_name)   # Создаем новый столбец с категоризацией по цели кредита 

df['purpose_category'].unique() # Смотрю уникальные значения

array(['приобретение объекта недвижимости', 'приобретение автомобиля',
       'кредит на образование', 'кредит на свадьбу', 'кредит на ремонт'],
      dtype=object)

#### Категоризация по количеству детей

In [12]:
def childrens(quantity):  # Сосздаем фунцию для категоризации по количеству детей.
    if quantity == 0:
        return 'Нет детей'
    if quantity == 1:
        return 'Один ребенок'
    if quantity == 2:
        return 'Двое детей'
    if quantity > 2:
        return 'Многодетные'

df['children_category'] = df['children'].apply(childrens) #Создам новый стобец с категоризацией по количеству детей

#### Создаем словарь для определениия наличия/отсутствия задолженности по кредиту

In [13]:
debt_vocabulary = {
    1:'Есть задолженности по возврату кредита',
    0:'Нет задолженности по возврату кредита'
}

df['debt'] = df['debt'].replace(debt_vocabulary) #Применяем словарь для внесения изменений в колонку датафрейма


df.info() # Проверяю наличие пустых значений
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 17 columns):
index                    21454 non-null int64
children                 21454 non-null int64
days_employed            21454 non-null int64
dob_years                21454 non-null int64
education                21454 non-null object
education_id             21454 non-null int64
family_status            21454 non-null object
family_status_id         21454 non-null int64
gender                   21454 non-null object
income_type              21454 non-null object
debt                     21454 non-null object
total_income             21454 non-null int64
purpose                  21454 non-null object
lemmatize_purpose        21454 non-null object
total_income_category    21454 non-null object
purpose_category         21454 non-null object
children_category        21454 non-null object
dtypes: int64(7), object(10)
memory usage: 2.8+ MB


**Вывод** Для ответа на поставленные вопросы нам потребуются категоризированные данные по категориям Доход, Цель кредита, Количество детей. Для удобства формирования сводных таблиц в отчете создаем для категорий отдельные столбцы.
Категоризацию в столбце Доход производил относительно медианного дохода. Соответственно взяв нижнюю границу медианный доход минус 50% такового, а верхнюю соответственно + 50%.
Категоризацию по цели кредита производил на основе значений, полученных методом Counter при лемматизации. Последовательность применения условий выставлял из соображений, что если в строке "ремонт жилья" сначала проверить на ключевое слово "жилье" - получу увеличение количество кредитов на жилье за счет отсутствия кредитов на ремонт.
Категоризацию по количеству детей производил основываясь на том, что 3 и более - это многодетные семьи, остальных разделил на группы "нет детей", "один ребенок", "двое детей"
Для дальнейшего формирования таблиц удобнее будет в столбце debt по задолженности произвести присвоения сотояний, для чего создаем словарь и заменяем значения в столбце.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
childrens_table = pd.pivot_table(df, index = ['children_category'],  columns=['debt'],  values = ['children'], aggfunc='count')# Создаю сводную таблицу, используя созданный категоризированный столбец

childrens_table['children','% невозвратов/задержек платежей'] = childrens_table['children','Есть задолженности по возврату кредита'] * 100 / (childrens_table['children','Есть задолженности по возврату кредита'] + childrens_table['children','Нет задолженности по возврату кредита']) #Во вновь созданную таблицу добавляю значение процента невозвратов по каждой группе, для наглядности отображения присвоив столбцу название на кириллице

childrens_table.sort_values(by=('children','% невозвратов/задержек платежей')) #Выводим таблицу

children  \
debt              Есть задолженности по возврату кредита   
children_category                                          
Нет детей                                           1063   
Многодетные                                           39   
Один ребенок                                         445   
Двое детей                                           194   

                                                         \
debt              Нет задолженности по возврату кредита   
children_category                                         
Нет детей                                         13028   
Многодетные                                         417   
Один ребенок                                       4410   
Двое детей                                         1858   

                                                   
debt              % невозвратов/задержек платежей  
children_category                                  
Нет детей                                7.543822  
Многодетные                              8.552632  
Один ребенок                             9.165808  
Двое детей                               9.454191

**Вывод** По результатам исследований можно сказать, что наименьший процент невозвратов/задержек платежей у заемщиков без детей. 
Второй по величине процент невозвратов/задержек платежей - у многодетных. 
На третьей позиции - У имеющих одного ребенка
Ну и максимальный процент невозвратов/задержек платежей у заемщиков с двумя детьми 
Из всего вышеизложенного следует, что наиболее адекватно свои возможности расчитывают многодетные, поскольку те у кого нет детей (при прочих равных) имеют бОльшие финансовые возможности. Семьи же с одни и двумя детьми имеют практически одинаковый процент.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
family_table = pd.pivot_table(df, index = ['family_status'],  columns=['debt'],  values = 'index', aggfunc='count') # Создаю сводную таблицу, используя созданный категоризированный столбец

family_table['% невозвратов/задержек платежей'] = family_table['Есть задолженности по возврату кредита'] * 100 / (family_table['Есть задолженности по возврату кредита'] + family_table['Нет задолженности по возврату кредита']) #Во вновь созданную таблицу добавляю значение процента невозвратов по каждой группе, для наглядности отображения присвоив столбцу название на кириллице

family_table.sort_values(by='% невозвратов/задержек платежей') #Выводим таблицу

debt,Есть задолженности по возврату кредита,Нет задолженности по возврату кредита,% невозвратов/задержек платежей
family_status,,,
вдовец / вдова,63,896,6.569343
в разводе,85,1110,7.112971
женат / замужем,931,11408,7.545182
гражданский брак,388,3763,9.347145
Не женат / не замужем,274,2536,9.750890


**Вывод** Наименьший процент невозвратов/задержек платежей у овдовевших, чуть больше у тех, кто находится в разводе. Самая рИсковая категория заемщиков это те кто Не женат / не замужем и с чуть меншим процентом те, кто находится в гражданском браке.
Более надежные заемщики - овдовевшие, находящиеся в разводе и женатые/замужние

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
table_income = pd.pivot_table(df, index = ['total_income_category'],  columns=['debt'],  values = 'total_income', aggfunc='count')

table_income['% невозвратов/задержек платежей'] = table_income['Есть задолженности по возврату кредита'] * 100 / (table_income['Есть задолженности по возврату кредита'] + table_income['Нет задолженности по возврату кредита']) #Во вновь созданную таблицу добавляю значение процента невозвратов по каждой группе, для наглядности отображения присвоив столбцу название на кириллице

table_income.sort_values(by='% невозвратов/задержек платежей')

debt,Есть задолженности по возврату кредита,Нет задолженности по возврату кредита,% невозвратов/задержек платежей
total_income_category,,,
низкий,93,1236,6.997743
высокий,341,4483,7.068823
без дохода,170,1933,8.083690
средний,1137,12061,8.614942


**Вывод** До выделения в отдельную категорию лиц без дохода таблица давала наиболее предсказуемый вывод - чем выше доход, тем меньше процент невозвратов/задержек платежей. Однако после выделения лиц без дохода в отдельную категорию ситуация изменилась - и наименьший процент невозвратов/задержек  приходится на заемщиков с низким доходом. 
А процент невозвратов/задержек у заемщиков без дохода скорее говорит о том, что они относятся к достатку чуть выше среднего, а 0-е значения в доходе - о его неофициальности или ошибке.
Заемщики с низким доходом к вопросу погашения кредитов относятся более скурпулезно.

- Как разные цели кредита влияют на его возврат в срок?

In [17]:
target_table = pd.pivot_table(df, index = ['purpose_category'],  columns=['debt'],  values = 'index', aggfunc='count')

target_table['% невозвратов/задержек платежей'] = target_table['Есть задолженности по возврату кредита'] * 100 / (target_table['Есть задолженности по возврату кредита'] + target_table['Нет задолженности по возврату кредита']) #Во вновь созданную таблицу добавляю значение процента невозвратов по каждой группе, для наглядности отображения присвоив столбцу название на кириллице

target_table.sort_values(by='% невозвратов/задержек платежей')

debt,Есть задолженности по возврату кредита,Нет задолженности по возврату кредита,% невозвратов/задержек платежей
purpose_category,,,
кредит на ремонт,35,572,5.766063
приобретение объекта недвижимости,747,9457,7.320659
кредит на свадьбу,186,2138,8.003442
кредит на образование,370,3643,9.220035
приобретение автомобиля,403,3903,9.359034


**Вывод** Наиболее рискованным является кредит на приобретение автомобиля, а наименее рискованным - на ремонт

## Шаг 4. Общий вывод

В предоставленных данных содержится очень много ошибок  в колонке days_employed (Стаж)(особенно это касается отрицательного стажа и непомерно большого стажа) - обратил бы внимание заказчика на проблему выгрузки или заполнения данных. В данном исследовании эти данные не применялись и были несущественны, в дальнейшем же все может измениться.
Также запросил бы - что означает неуказанный доход?  

Основываясь на проведенных проведенных исследованиях можно сделать вывод, что наиболее надежный заемщик это заемщик с высоким достатком, не имеющий семьи (но имевший таковую) и детей и берущий кредит на ремонт (хотя в таком варианте кредит на ремонт больше похож на утопию), скорее уж кредит на недвижимость.
Наиболее рИсковый вариант - это не женатые/не замужем со средним доходом берущие кредит на приобретение автомобиля. С другой стороны, если ставится задача получить дополнительный доход в виде штрафных санкций - для данного варианта не самый плохой вариант.
При построении модели кредитного скоринга особое внимание обратил бы на группу заемщиков семейных со средним доходом, они хотя и являются группой с рисками выше среднего, но это самая многочисленная группа.